# Data Analysis Assignment 4
**Group: Ohm_Squad**

**Members: Rauch,Bilijesko,Frizberg**

**Datasets: Westermo**

## Initial Setup

In [23]:
# Initial setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
import os

# Configure plotting
plt.rcParams.update({
    'figure.figsize': [12, 8],
    'figure.dpi': 150,
    'figure.autolayout': True,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'font.size': 12
})

pathRaw = "./data_raw/"
pathFilter = "./data_filtered/"
pathProcessd = "./data_processed/"
pathVisuRaw = "./visu_raw/"
pathVisuProcessed = "./visu_processed/"
pathOnlyProcessed = "./visu_only_processed/"
pathProb = "./visu_prob/"

files = [f"system-{number}.csv" for number in range(1, 20)]

# Systems 3, 5, 6, 8, 11 and 17 do not have sys-thermal readings ! 3/5/6 -> crashes 8/11/17 -> no thermal
remove_entries = [7,10,16]
files = [item for index, item in enumerate(files) if index not in remove_entries]
# --------------------- chosen for analysis ---------------------------------------------------------------------- 
# 19
present = files[-1]

sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.2)

np.random.seed(42)

# 2.1 Data Preprocessing and Basic Analysis
- **Basic statistical analysis using pandas**
> -> see load_system_data()
- **Original data quality analysis (including visualization)**
> -> see Analysis Notes after visu_raw_data()
- **Data preprocessing**
> -> see preprocess_system_data() and "data_processed"
- **Preprocessed vs original data visual analysis**
> -> see Analysis Notes after visu_processed_data()

# 2.2 Visualization and Exploratory Analysis
- **Time series visualizations**
- **Distribution analysis with histograms**
- **Correlation analysis and heatmaps**
- **Daily pattern analysis**
> -> see visu_processed_data() and Analysis Notes after visu_processed_data()
- **Summary of observed patterns - similar to True/False questions**
> -> see Analysis Notes after visu_processed_data()

>All figures/plots can be accessed in "visu_raw", "visu_processed" and "visu_only_processed".



.

## Loading and Filtering
Files are fetched from directory and prefiltering for columns of interst.

Processing timestamps to datetime for usage in timeseries (and usability).

Done via a function to execute for every file separately and be able to pipe if necessary.

Returning the dataframe could be either dropped or caught by either a container or piped into the next function.

- **2.3: Basic statistical analysis using pandas**
>  -> output into CSV (visu_raw)


In [24]:
def load_system_data(file_dir: str, file_name: str) -> pd.DataFrame :
    """Load and prepare test system performance data.
    
    Parameters
    ----------
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    
    Additional outputs
    saves filtered data into dir "./data_filtered"
    
    Returns
    -------
    pd.DataFrame
        Raw dataframe with columns:
        - datetime (index)
        - load-15m
        - memory_used_pct
        - cpu-user
        - cpu-system
        - sys-thermal
        - sys-interrupt-rate
        - server-up
        - disk-io-time
    """
    file_path = file_dir + file_name

    df = pd.read_csv(file_path, delimiter = ",",usecols=["timestamp",
                                                         "load-15m",
                                                         "sys-mem-available",
                                                         "sys-mem-total",
                                                         "cpu-user",
                                                         "cpu-system",
                                                         "sys-thermal",
                                                         "sys-interrupt-rate",
                                                         "server-up",
                                                         "disk-io-time"]) # Read in data with columns
    

    
    df['datetime'] = pd.to_datetime(df['timestamp'], unit = 's', errors = 'coerce') # Create datetime from timestamp
    
    df.set_index('datetime', inplace=True) # Set datetime as index

    df['memory_used_pct'] = (1 - df['sys-mem-available']/df['sys-mem-total']) * 100 # Memory usage calculation
    df.drop(["timestamp","sys-mem-available","sys-mem-total"], axis=1, inplace=True) # Drop unneccessary data
    
    df.to_csv(pathFilter+file_name, index=True) # output filtered .csv to path (dir)
    
    file_name = file_name.replace(".csv","")
    df.describe().to_csv(f'{pathVisuRaw}{file_name}_desciption.csv') # output pandas desscribe() result into .csv to path (dir)
    
    return df

In [25]:
# testing 
df = load_system_data(pathRaw,present)

In [26]:
# Pre filter all files
# for file in files:
#     load_system_data(pathRaw, file)

## Visualizing Raw
- **2.1: Original data quality analysis (including visualization)**
- **2.2: Time series visualizations**
- **2.2: Distribution analysis with histograms**
- **2.2: Correlation analysis and heatmaps**
- **2.2: Daily pattern analysis**

First: Helper functions for interacting with images and os to delete temporary files.

Second: Main function for visualizing

In [27]:
# adapted https://stackoverflow.com/questions/6996603/how-can-i-delete-a-file-or-folder-in-python
def delete_images(files: list) -> None:
    """Deletes the files specified in the list of file paths.
    Parameters
    ----------
    files: list[str]
        List of names of image files to put into .pdf file. 
    
    Additional output
    ----------
        Deltes list of images.

    Returns
    -------
        None
    """
    for file in files:
        try:
            if os.path.exists(file):
                os.remove(file)
                #print(f"Deleted: {file}")
            else:
                print(f"File not found: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")
            
# adapted https://stackoverflow.com/questions/40906463/png-images-to-one-pdf-in-python 
# and https://www.geeksforgeeks.org/save-multiple-matplotlib-figures-in-single-pdf-file-using-python/ 
def save_image(image_names: list, out_dir: str, filename: str) -> None: 
    """Gathers multiple plt.figure obejcts and outputs thm into a .pdf 
    
    Parameters
    ----------
    image_names: list[str]
        List of names of image files to put into .pdf file   
    out_dir: str
        Path to the directory of output .pdf file
    filename: str
        Name of output .pdf file
        
    Additional output
    ----------
    Saves a .pdf created by multiple .pngs into specified directory

    Returns
    -------
    None
    """
    image_list = [] #contains opened files
    for name in image_names:
        print(name)
        image_list.append(Image.open(name))

    image_list[0].save(f"{out_dir}{filename}_allPlots.pdf", save_all=True, append_images=image_list[1:])
    for image in image_list:
        image.close()
    print(f"{out_dir}{filename}_allPlots.pdf")
    delete_images(image_names)

In [28]:
def visu_raw_data(show_plots: bool, file_dir: str, file_name: str, df_arg: pd.DataFrame, isRaw: bool = True):
    """Load and visualize filtered and processed test system performance data.
    
    Parameters
    ----------
    show_plots: bool
        Just output files or display in notebook
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    isRaw : bool (Default: True)
        function can be used to visualize any raw or processed -> changes data_type (string) and out_dir (string)
        
    optional
    df_arg: pd.DataFrame
        output from load_system_data()

    Additional output
    ----------
    saves visualized data into dir "./visu_raw" by calling save_image() and cleaning temp-files with delete_images()
    
    Returns
    -------
    None
    """
    # Check DataFrame was passed
    if isinstance(df_arg, pd.DataFrame):
        df = df_arg
        # File name and path -> pd used => no identifier => using "./" 
        out_dir = "./"
        out_name = "Visu_output_noident"
        print("Function called with a DataFrame.")
    else:
        # Attempt to read the DataFrame from file
        try:
            file_path = file_dir + file_name
            df = pd.read_csv(file_path, delimiter = ",",usecols=["datetime","load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","server-up","disk-io-time"])
            print(f"Function called with a file: {file_path}")
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)
            # File name and path -> path used => use identifier 
            out_dir = pathVisuRaw
            out_name = file_name.replace('.csv', '')
        except Exception as e:
            print(f"Error loading the file: {e}")
            return None
        
    measurements = {
        "load-15m": ('load-15m', '%'),
        "memory_used_pct": ('memory_used_pct', '%'),
        "cpu-user": ('cpu-user', 'delta-s'),
        "cpu-system": ('cpu-system', 'delta-s'),
        "sys-thermal": ('sys-thermal', 'avg delta-°C/min'),
        "sys-interrupt-rate": ('sys-interrupt-rate', 'delta-s'),
        "disk-io-time": ('disk-io-time', 'delta-s')
        #,"server-up": ('server-sup', '')
    }
    if (isRaw):
        data_type = "Raw"
    else:
        data_type = "Processed"
        out_dir = pathOnlyProcessed
    
    image_names = []
    image_nr = 0
    
    # Plot 1: Time-Series
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle(f"Tme-Series - {data_type} Data", fontsize=16, y=1.02)
   
    for i,(measure, (title, unit)) in enumerate(measurements.items()): # iterate over all measurements
        row = i // 2
        col = i % 2

        df.iloc[::10].pivot(columns='server-up', values=measure).plot(ax=axes[row, col],alpha=0.7, linewidth=2,color=['red','blue']) # only use every 10th entry

        axes[row, col].set_title(f'Time-Series of {measure.upper()}')
        axes[row, col].set_xlabel('Datetime')
        axes[row, col].set_ylabel(f'{title} ({unit})')
        axes[row, col].grid(True)
        axes[row, col].legend()
        
    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 2: Daily Patterns
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle(f"Daily Patterns of {data_type} Measurements - mean & std ", fontsize=16, y=1.02)

    # Create hour column for grouping
    df_hour = df.copy()
    df_hour['hour'] = df_hour.index.hour # set index to hour (of day)

    
    for i, measurement in enumerate(measurements):
        row = i // 2
        col = i % 2
        
        # Calculate hourly statistics
        hourly_stats = df_hour.groupby('hour')[measurement].agg(['mean', 'std'])
        
        # Plot mean with standard deviation
        axes[row, col].plot(hourly_stats.index, hourly_stats['mean'], 'b-', label='Mean')
        axes[row, col].fill_between(
            hourly_stats.index,
            hourly_stats['mean'] - hourly_stats['std'],
            hourly_stats['mean'] + hourly_stats['std'],
            alpha=0.2,
            label='±1 std'
        )
        
        axes[row, col].set_title(f'Daily {measurement.capitalize()} Pattern')
        axes[row, col].set_xlabel('Hour of Day')
        axes[row, col].set_ylabel(measurement)
        axes[row, col].grid(True)
        axes[row, col].legend()


    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 3: Hour-wise Distributions
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle(f" {data_type} Measurement Distributions by Hour - Boxplots", fontsize=16, y=1.02)
        
    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2

        #calculates and plots hou wise boxplot
        df_hour.boxplot(
            ax=axes[row, col],
            column=measure,
            by='hour'
        )
        axes[row, col].set_title(f'Daily Pattern of {title} ')
        axes[row, col].set_xlabel('Hour of Day')
        axes[row, col].set_ylabel(f'{title} ({unit})')
        axes[row, col].grid(True)

    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 4 Histograms - Distribution
    fig, axes = plt.subplots(4,2, figsize = (15, 25))
    fig.suptitle(f"Sensor {data_type} Measurements Distributions", fontsize = 14)

    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2
        bin_num = 50 # was determined by trial an error
        
        # 200 bins are created for each measurement 
        axes[row, col].hist(df[measure], bins = bin_num*4, density = True, alpha = 0.7, label = 'Histogram')
        axes[row, col].set_title(f'Distribution of {title} ')
        axes[row, col].set_xlabel( f'{title} ({unit})')
        axes[row, col].set_ylabel('Density')
        axes[row, col].grid(True)
        
        #second axis for line graph
        # 50 points for the line are used better average behaviour
        ax_2 = axes[row, col].twinx()
        counts, bins = np.histogram(df[measure], bins = bin_num)
        bin_centers = (bins[:-1] + bins [1:]) / 2
        ax_2.plot(bin_centers, counts/counts.sum(), 'r-', lw = 2, label = 'Distribution')        
        ax_2.tick_params(axis='y', labelcolor='r')
        ax_2.legend()

    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 5: Correlation Analysis
    fig, (ax) = plt.subplots(1, 1, figsize=(15, 10))
    fig.suptitle(f"Correlation Analysis - of {data_type} Measurements Correlations", y=1.02, fontsize=16)

    # Correlations heatmap
    sns.heatmap(
        df[measurements.keys()].corr(),
        annot=True,
        cmap='coolwarm',
        center=0,
        fmt='.2f',
        ax=ax
    )
    
    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    # Plot 6 Hexbins    
    measure = list(measurements.keys()) #["load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","disk-io-time"] 
    pairs = [(measure[i], measure[j]) for i in range(len(measure)) for j in range(i + 1, len(measure))] # https://www.w3schools.com/python/python_lists_comprehension.asp

    n_rows = 7
    n_cols = 3

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    fig.suptitle(f"Hexbins of {data_type} Measurements", y=1.02, fontsize=16)
    axes = axes.flatten()

    # Loop over pairs
    for i, (measure1, measure2) in enumerate(pairs):
        ax = axes[i]
        x = df[measure1]
        y = df[measure2]
        
        title1, unit1 = measurements[measure1]
        title2, unit2 = measurements[measure2]

        hb = ax.hexbin(x, y, gridsize=100, cmap='viridis')
        ax.set_xlabel(f'{title1} ({unit1})')
        ax.set_ylabel(f'{title2} ({unit2})')
        ax.set_title(f'Hexbin: {title1} vs {title2}')
        
        fig.colorbar(hb, ax=ax)
        
    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 7: Scatter Matrix
    # Get data without duplicates by taking mean for each timestamp
    df_plot = df.groupby(df.index)[measure].mean()
    try:
        pp = sns.pairplot(data=df_plot,
                            diag_kind='kde',
                            plot_kws={'alpha': 0.5, 's': 20},
                            height = 3,
                            corner=True)
    except Exception as e:
        print(f"Warning: Could not create scatter matrix plot: {str(e)}")

    fig = pp.figure
    fig.suptitle('Scatter Matrix of Raw Measurements', y=1.02, fontsize=16)
    
    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    save_image(image_names, out_dir, out_name)
    if not show_plots:
        plt.close("all")

In [29]:
# testing 
visu_raw_data(True, pathProcessd, present,None, False)

Function called with a file: ./data_processed/system-19.csv
./visu_only_processed/system-19_plot_0.png
./visu_only_processed/system-19_plot_1.png
./visu_only_processed/system-19_plot_2.png
./visu_only_processed/system-19_plot_3.png
./visu_only_processed/system-19_plot_4.png
./visu_only_processed/system-19_plot_5.png
./visu_only_processed/system-19_plot_6.png
./visu_only_processed/system-19_allPlots.pdf


In [30]:
#Run Visualization of Raw
# for file in files:
#     visu_raw_data(False, pathFilter,file,None)
#     plt.close("all") #for safety

## Analysis
- **2.1: Original data quality analysis (including visualization)**
> - Has spikes (outliers) especially bad one in sys-interrupt and CPU-user
> - Includes Nan- or empty-entries
> - large hour-wise distributions due to the outliers

## Processing

- thresholds- and IQR-method
- aggregation

In [16]:
def remove_outliers_iqr(show_process_status: bool, df:pd.DataFrame, column: str) -> tuple:
    """Remove outliers using IQR method. 
    Why this method: https://builtin.com/articles/1-5-iqr-rule 

    Parameters
    ----------
    show_process_status : bool
        print status in console
    df : pd.DataFrame
        input data for cleaning
    column: str
        current column to "look at"

    Returns
    -------
        (pd.Series, pd.Series)
            cleaned data , outliers
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    # decision range: lower & upper bound
    # (*1.5) -> rule comes closest to Gaussian distribution concerning the outlier detection (*1.7 = 3 sigma)
    valid_mask = (df[column] >= Q1 - 1.5*IQR) & (df[column] <= Q3 + 1.5*IQR)
    invalid_count = (~valid_mask).sum()
    if show_process_status:
        print(f"IQR: Removing {invalid_count} outliers from {column}")
    return df[column].where(valid_mask, np.nan), df[column].where(~valid_mask)

def handle_missing_values(data: pd.DataFrame, column: str, max_gap: int = 8) -> pd.Series: # not used
    """Interpolate missing values with limit.
    Parameters
    ----------
    data : pd.DataFrame
        _description_
    column : str
        _description_
    max_gap: int, optional
        _description_ (Defaults to 8.)

    Returns
    ----------
    pd.Series : 
        _description_
    """
    return data[column].interpolate(
        method='linear',
        limit=max_gap  # Only fill gaps up to 8 points
    )

def preprocess_system_data(show_process_status: bool, file_dir: str, file_name: str, df_arg: pd.DataFrame = None) -> list:
    """Preprocess system performance data.
    Cleans data with:
          * Invalid values removed
          * Duplicates handled
          * Outliers removed
          * Missing values interpolated
    
    Parameters
    ----------
    show_process_status: bool
        ...
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    
    optional
    df_arg: pd.DataFrame
        output from load_system_data()

    Returns
    -------
    df_original
        ...
    df_cleaned
        ...
    
    str: filename
        ...
    """
    # Check DataFrame was passed
    if isinstance(df_arg, pd.DataFrame):
        df = df_arg
        print("Function called with a DataFrame.")
    else:
        # Attempt to read the DataFrame from file
        try:
            file_path = file_dir + file_name
            df = pd.read_csv(file_path, delimiter = ",",usecols=["datetime","load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","server-up","disk-io-time"])
            print(f"Function called with a file: {file_path}")
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)
        except Exception as e:
            print(f"Error loading the file: {e}")
            return None
    # Store original data
    df_original = df.copy()
    df_outliers = df.copy()
    out_dir = pathProcessd
    
    valid_ranges = {
        "load-15m":  (0, 1.0), 
        "memory_used_pct": (0, 100),
        "cpu-user": (0.0, 2.0),
        "cpu-system": (0.0, 2.0),
        "sys-thermal": (-10, 10),
        "sys-interrupt-rate": (0, 100000),
        "disk-io-time": (0, 1.0),
        "server-up": (0, 2)
    }
    columns = list(valid_ranges.keys())
    
    # 1. Handle invalid values
    #(thresholds for sensible bounds determined by examining raw datasets)
    for column, (min_val, max_val) in valid_ranges.items():
        invalid_mask = (df[column] < min_val) | (df[column] > max_val)
        if show_process_status:
            print(f"Ranges: Removing {invalid_mask.sum()} invalid values from {column}")
        df.loc[invalid_mask, column] = np.nan
    
    # 2. Handle duplicates
    if show_process_status:
        print("Handling duplicates ...")
    df = df.groupby(['datetime', 'server-up']).agg({ #Groups df by combination of: datetime and server-up
        'load-15m': 'mean',
        'memory_used_pct': 'mean',
        'cpu-user': 'mean',
        "cpu-system": 'mean',
        'sys-thermal': 'mean' ,
        "sys-interrupt-rate": 'mean',
        "disk-io-time": 'mean'
    }).reset_index()
    #Result: row unique combination of datetime and server-up -> aggregated by mean (2.5 | 3.5 -> 3.0)
        
    # 3. Remove outliers
    for column in columns:
        df[column],df_outliers[column] = remove_outliers_iqr(show_process_status, df, column)
    # testing df.to_csv("noHandling_data.csv", index=False)
    
    # 4. Handle missing values <-- just delete rows with empty entries -> no interpolation ! 
    # there is ennough data and interpolating seemed like 
    #   misrepresentation of data characteristics and 
    #   as bias introduction
    '''if show_process_status:
        print("\nHandling missing values...")
        print(f"Missing values before handling: \n{df.isnull().sum()}")
    for  column, (min_val, max_val) in valid_ranges.items():
         df[column] = handle_missing_values(df, column,4)'''
    
    df_cleaned = df.dropna()
    df_cleaned.set_index('datetime', inplace=True)
    # testing print("After dropping empty entries: \n", df_cleaned.head())
    
    # Sort by datetime
    df_cleaned.sort_index(inplace=True)
    
    if show_process_status:
        print(f"Missing values after handling: \n{df_cleaned.isnull().sum()}")
        print(f"\nOriginal shape: {df_original.shape}")
        print(f"Cleaned shape: {df_cleaned.shape}")
    
    df_cleaned.to_csv(out_dir+file_name, index=True)

    return [df_original, df_cleaned, file_name]

In [19]:
# tesing
preprocess_system_data(True, pathFilter,present,None)

Function called with a file: ./data_filtered/system-19.csv
Ranges: Removing 0 invalid values from load-15m
Ranges: Removing 0 invalid values from memory_used_pct
Ranges: Removing 0 invalid values from cpu-user
Ranges: Removing 0 invalid values from cpu-system
Ranges: Removing 0 invalid values from sys-thermal
Ranges: Removing 0 invalid values from sys-interrupt-rate
Ranges: Removing 0 invalid values from disk-io-time
Ranges: Removing 0 invalid values from server-up
Handling duplicate timestamps...
IQR: Removing 2690 outliers from load-15m
IQR: Removing 284 outliers from memory_used_pct
IQR: Removing 12113 outliers from cpu-user
IQR: Removing 12765 outliers from cpu-system
IQR: Removing 26 outliers from sys-thermal
IQR: Removing 8282 outliers from sys-interrupt-rate
IQR: Removing 2732 outliers from disk-io-time
IQR: Removing 6 outliers from server-up

Handling missing values...
Missing values before handling: 
datetime                  0
server-up                 6
load-15m             

[                     load-15m  sys-interrupt-rate  sys-thermal  disk-io-time  \
 datetime                                                                       
 1970-01-01 00:00:00      0.02             1322.05         0.00        0.0002   
 1970-01-01 00:00:30      0.01             1236.20         1.40        0.0002   
 1970-01-01 00:01:00      0.01             1266.25         0.00        0.0002   
 1970-01-01 00:01:30      0.01             1248.10         1.45        0.0002   
 1970-01-01 00:02:00      0.01             1290.00         1.45        0.0002   
 ...                       ...                 ...          ...           ...   
 1970-01-30 23:58:00      0.00             1267.25         1.45        0.0006   
 1970-01-30 23:58:30      0.00             1266.45         1.50        0.0002   
 1970-01-30 23:59:00      0.00             1354.75         1.50        0.0004   
 1970-01-30 23:59:30      0.00             1276.20         1.55        0.0006   
 1970-01-31 00:00:00      0.

In [10]:
# Run processing
# anylist = []
# for file in files:
#   anylist = preprocess_system_data(False, pathFilter,file,None)
#   print(anylist[0], " : \n", anylist[1].describe(),"\n", anylist[2].describe()) 

Error loading the file: [Errno 2] No such file or directory: './data_filtered/s'


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Test: if processed data makes sense 
#cache_list = preprocess_system_data(True, pathFilter,"system-1.csv",None)
#visu_raw_data(True, None,None,pd.DataFrame(cache_list[1]))

In [ ]:
#Run Visualization of Processed
# for file in files:
#     visu_raw_data(False, pathProcessd,file,None, False)
#     plt.close("all") #for safety

## Loadfile

- in case different sets shall be compared. 
- if data is to be loaded into a dataframe instead of directly accessed by a function.
- Otherwise visu_processed_data will be called directly after preprocess_system_data(). Since their IOs are suitable.

In [ ]:
def load_file(file_dir: str, file_name: str, create_description: bool = False) -> tuple:
    """Loads file from path and returns dataframe and its name (as tuple).
    
    Parameters
    ----------
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    create_description: bool

    
    Additional output
    ----------
    pandas.description of passed dataset into the directory the file was called from.
    
    Returns
    --------
    tuple(pd.DataFrame, str)
    pd.DataFrame: _description_
        ...
    str: file_name
        ...
    """
    try:
        file_path = file_dir + file_name
        df = pd.read_csv(file_path, delimiter = ",",usecols=["datetime","load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","server-up","disk-io-time"])
        print(f"Function called with a file: {file_path}")
        df['datetime'] = pd.to_datetime(df['datetime'])
        df.set_index('datetime', inplace=True)
        # File name and path -> path used => use identifier
        file_name = file_name.replace('.csv', '')
        if create_description:
            df.describe().to_csv(f'{file_dir}{file_name}_desciption.csv')
        return (df, file_name)
    except Exception as e:
        print(f"Error loading the file: {e}")
        return (None,None)
    

#testing
#print(load_file(pathFilter, files[0]))

In [ ]:
# creating data description for all processed datasets.
# for file in files:
#     load_file(pathProcessd,file, True)
#     plt.close("all") #for safety

## Visualize Raw & Processed
- **2.2: Time series visualizations**
- **2.2: Distribution analysis with histograms**
- **2.2: Correlation analysis and heatmaps**
- **2.2: Daily pattern analysis**

In [21]:
def visu_processed_data(show_plots: bool, df_original: pd.DataFrame, df_cleaned: pd.DataFrame, filename: str) -> None:
    """visualize original and processed test system performance data.
    
    Parameters
    ----------
    show_plots: bool
        Just output files or display in notebook
    df_cleaned: pd.DataFrame
        ...
    df_original: pd.DataFrame
        ...
    filename: str
        ... for pdf output

    Additional outputs
    saves visualized data into dir "./visu_processed" by calling save_image() and cleaning temp-files with delete_images()
    
    Returns
    -------
        None
    """
    
    out_dir = pathVisuProcessed
    out_name = filename.replace('.csv','')
    image_names = []
    image_nr = 0
   
    measurements = {
        "load-15m": ('load-15m', '%'),
        "memory_used_pct": ('memory_used_pct', '%'),
        "cpu-user": ('cpu-user', 'delta-s'),
        "cpu-system": ('cpu-system', 'delta-s'),
        "sys-thermal": ('sys-thermal', 'avg delta-°C/min'),
        "sys-interrupt-rate": ('sys-interrupt-rate', 'delta-s'),
        "disk-io-time": ('disk-io-time', 'delta-s')
        #,"server-up": ('server-sup', '')
    }
    measures = list(measurements.keys())

    # Plot 1: Time-Series
    # see general system behaviour and easy to spot anomalies 
    # e.g. data-error -> blank ... or system-3: sucessive rise in memory_usage would suggest a memeory-leak
    # system-19 shows a clear problem around 16-17.01. visible in load-15m, memory_used and best visible in sisk-io (processed data)
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle('Tme-Series - Raw Data', fontsize=16, y=1.02)

    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2
        df_original[measure].iloc[::10].plot(ax=axes[row, col], color='lightblue', alpha=0.3, label='Original')
        df_cleaned[measure].iloc[::10].plot(ax=axes[row, col], color='green', alpha=0.5, label='Cleaned')
        axes[row, col].set_title(f'Time-Series of {measure.upper()}')
        axes[row, col].set_xlabel('Datetime')
        axes[row, col].set_ylabel(f'{title} ({unit})')
        axes[row, col].grid(True)
        axes[row, col].legend()
        
    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 2: Daily Patterns
    # shows hourly behaviour (mean and std) overlaying original / processed shows better (thighter) std 
    # but also might suggest loss of relevant data by processing: cpu-system behaviour and sys-interrupt
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle('Daily Patterns of Raw & Processed Measurements - mean & std ', fontsize=16, y=1.02)

    # Create hour column for grouping
    df_hour_orig = df_original.copy()
    df_hour_clean = df_cleaned.copy()
    df_hour_orig['hour'] = df_hour_orig.index.hour
    df_hour_clean['hour'] = df_hour_clean.index.hour
    
    for i, measurement in enumerate(measurements):
        row = i // 2
        col = i % 2
        
        # Calculate hourly statistics
        hourly_stats_orig = df_hour_orig.groupby('hour')[measurement].agg(['mean', 'std'])
        hourly_stats_clean = df_hour_clean.groupby('hour')[measurement].agg(['mean', 'std'])

        # Plot mean with standard deviation
        axes[row, col].plot(hourly_stats_clean.index, hourly_stats_clean['mean'], 
                        'g-', label='Mean Processed')
        axes[row, col].fill_between(
            hourly_stats_clean.index,
            hourly_stats_clean['mean'] - hourly_stats_clean['std'],
            hourly_stats_clean['mean'] + hourly_stats_clean['std'],
            alpha=0.3,
            color='lightgreen',
            label='±1 std Processed'
        )

        axes[row, col].plot(hourly_stats_orig.index, hourly_stats_orig['mean'], 
                        'b-', label='Mean Raw')
        axes[row, col].fill_between(
            hourly_stats_orig.index,
            hourly_stats_orig['mean'] - hourly_stats_orig['std'],
            hourly_stats_orig['mean'] + hourly_stats_orig['std'],
            alpha=0.2,
            color='lightblue',
            label='±1 std Raw'
        )
        #ax_2.tick_params(axis='y', labelcolor='b')
        
        axes[row, col].set_title(f'Daily {measurement.capitalize()} Pattern')
        axes[row, col].set_xlabel('Hour of Day')
        axes[row, col].set_ylabel(measurement)
        axes[row, col].grid(True)
        axes[row, col].legend()


    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    del df_hour_clean, df_hour_orig

    # Plot 4 Histograms - Distribution (only original data)
    # how is data distibuted, are ther multiple peaks (accumulations), 
    # problematic sice every system and every metric would need their specific amount of bins and averaging-line 
    # 50/200 was deemed the best compromise => most figures suggest normal distributions (sigular and multimodal)
    # in case of system-19: 
    #   cpus-user &-system aswell as sys-interrupt are normal distributed. 
    #   load-15m (bad bin-amount), memory_used and sys-thermal are multi-modla 
    #   disk-io looks to be multimodal, but in this systems-case there might be a problem (low-res. sensor or bad software readout) 
    #       since compared to e.g. system-4 (two peaks) the datapoints are in suspiciously periodic (consistant) distnaces (0.002 delta-s)
    
    fig, axes = plt.subplots(4,2, figsize = (15, 25))
    fig.suptitle('Sensor Processed Measurements Distributions', fontsize = 14)

    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2
        bin_num = 50
        axes[row, col].hist(df_cleaned[measure], bins = bin_num*4, density = True, alpha = 0.7)
        axes[row, col].set_title(f'Distribution of {title} ')
        axes[row, col].set_xlabel( f'{title} ({unit})')
        axes[row, col].set_ylabel('Density')
        axes[row, col].grid(True)
        
        #second axis for line graph
        ax_2 = axes[row, col].twinx()
        counts, bins = np.histogram(df_cleaned[measure], bins = bin_num)
        bin_centers = (bins[:-1] + bins [1:]) / 2
        ax_2.plot(bin_centers, counts/counts.sum(), 'r-', lw = 2, label = 'Distribution')        
        ax_2.tick_params(axis='y', labelcolor='r')
        ax_2.legend()



    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 5: Correlation Analysis
    # how does one metric correlate with another ? does cpu-user lead to high load-15m ?
    # highest correlations:     
    #   cpu-system(/-user) & sys-interrupts 
    #   cpu-system & -user (which is to be expected) 
    #   cpu-user & memory_used (also expected as a working cpu probably requests larger data outside its L1-3 caches)
    # sys-thermals show no correlation (counter-intuitive): assumed reason being that value is absolute temp-change -> hard to correlate cooling/heating 
    # also interesting that there is "low" correlation between memory_used and disk-io 
    #   -> if information is requested for a process (calculation) from disk it should be moved to RAM (system-4 correl. suggests this)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    fig.suptitle('Correlation Analysis - Original vs Cleaned', y=1.02, fontsize=16)

    # Original correlations
    sns.heatmap(
        df_original[measures].corr(),
        annot=True,
        cmap='Blues', #coolwarm
        center=0,
        fmt='.2f',
        ax=ax1
    )
    ax1.set_title('Original Data Correlations')

    # Cleaned correlations
    sns.heatmap(
        df_cleaned[measures].corr(),
        annot=True,
        cmap='Greens',
        center=0,
        fmt='.2f',
        ax=ax2
    )
    ax2.set_title('Cleaned Data Correlations')

    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 6 Hexbins
    # Visualizing dense (aggregating) data, see trends and clustering (relationships), color gradients make it easier to digest and interopret
    # poor results for some relations 
    pairs = [(measures[i], measures[j]) for i in range(len(measures)) for j in range(i + 1, len(measures))]

    n_rows = 7
    n_cols = 3

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    fig.suptitle('Hexbins of Processed Measurements', y=1.02, fontsize=16)
    axes = axes.flatten()

    for i, (measure1, measure2) in enumerate(pairs):
        ax = axes[i]
        x = df_cleaned[measure1]
        y = df_cleaned[measure2]
        
        title1, unit1 = measurements[measure1]
        title2, unit2 = measurements[measure2]
        
        hb = ax.hexbin(x, y, gridsize=100, cmap='viridis')
        ax.set_xlabel(f'{title1} ({unit1})')
        ax.set_ylabel(f'{title2} ({unit2})')
        ax.set_title(f'Hexbin: {title1} vs {title2}')
        fig.colorbar(hb, ax=ax)
        
    #---------------------------------------------- File output
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 7: Scatter Matrix
    # similar to hexbins -> was used to see clear distiction before and after processing
    df_plot1 = df_original
    df_plot1['State'] = 'raw'
    df_plot2 = df_cleaned
    df_plot2['State'] = 'processed'
    df_plot1 = pd.concat([df_plot1, df_plot2])
    
    #testing
    # df_plot1.to_csv(out_dir+"concatPlot1Plot2.csv", index=True)
    # print(df_plot1.shape, " and ", df_plot2.shape)
    # print(df_plot1.head(), "\n", df_plot2.head())
    
    del df_plot2
    pp = None
    try:
        pp = sns.pairplot(data=df_plot1,
                            diag_kind='kde',
                            vars = measures,
                            hue='State',
                            markers=["o","D"],
                            plot_kws={'alpha': 0.5, 's': 20},
                            height = 3,
                            corner=True)
    except Exception as e:
        print(f"Warning: {str(e)}")

    fig = pp.figure
    fig.suptitle('Scatter Matrix of Raw and Processed Measurements', y=1.02, fontsize=16)
    
    del df_plot1
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    save_image(image_names, out_dir, out_name)

    if not show_plots:
        plt.close("all")

In [22]:
# testing
# load same file filtered and processed

ca1 = load_file(pathFilter,present)
ca2 = load_file(pathProcessd,present)
#visualize it (exporting pdf)
visu_processed_data(True, ca1[0], ca2[0], ca2[1])

NameError: name 'load_file' is not defined

In [ ]:
# anylist = []
# for file in files:
#     anylist = preprocess_system_data(False, pathFilter,file,None)
#     visu_processed_data(False, anylist[0],anylist[1],anylist[2])
#     #visu_processed_data(False, preprocess_system_data(False, pathFilter,file,None))

## Analysis
- **2.1: Preprocessed vs original data visual analysis**
> ...
- 2. **Time Patterns**
   - Consider day vs night patterns
   - Maintain testing cycles
   - Preserve resource patterns

3. **Data Consistency**
   - Check metric relationships
   - Validate cleaning impact
   - Document changes

   1. Test Execution Patterns:\
a_) System load shows clear day/night testing cycles

2. System Performance:\
b_) High load periods (load-15m) align with increased Memory usage\
c_) High memory usage periods coincide with increased CPU usage rate of change\
d_) Temperature change rate increases during high system load periods

3. Resource Utilization:\
e_) Memory usage consistently increases at test start and gradually decreases towards test completion\
f_) System load stays within reasonable limits (<0.4) during normal operation

4. System Behavior:\
g_) Memory usage returns to idle state levels (around 5-6%) between test cycles\
h_) Load, memory, and CPU metrics collectively show clear patterns distinguishing between test execution and idle periods\

## 2.3 Probability Analysis
- **Threshold-based probability estimation**
- **Cross tabulation analysis**
- **Conditional probability analysis**
- **Summary of observations from each task**

In [ ]:
def prob_analysis(show_plots: bool, file_dir: str, file_name: str, df_arg: pd.DataFrame = None) -> None:
    """Prob analysis...
    
    Parameters
    ----------
    show_process_status: bool
        ...
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    
    optional
    df_arg: pd.DataFrame
        output from load_system_data()

    Additional outputs
    saves visualized data into dir "./visu_prob" by calling save_image() and cleaning temp-files with delete_images()
    
    Returns
    -------
    None
    """
    # Check DataFrame was passed
    if isinstance(df_arg, pd.DataFrame):
        df = df_arg
        out_dir = "./"
        out_name = "Visu_output_noident"
        print("Function called with a DataFrame.")
    else:
        # Attempt to read the DataFrame from file
        try:
            file_path = file_dir + file_name
            df = pd.read_csv(file_path, delimiter = ",",usecols=["datetime","load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","server-up","disk-io-time"])
            print(f"Function called with a file: {file_path}")
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)
            # File name and path -> path used => use identifier 
            out_dir = pathProb
            out_name = file_name.replace('.csv', '')
        except Exception as e:
            print(f"Error loading the file: {e}")
            return None
        
    image_names = []
    image_nr = 0

    measurements = {
        "load-15m": ('load-15m', '%'),
        "memory_used_pct": ('memory_used_pct', '%'),
        "cpu-user": ('cpu-user', 'delta-s'),
        "cpu-system": ('cpu-system', 'delta-s'),
        "sys-thermal": ('sys-thermal', 'avg delta-°C/min'),
        "sys-interrupt-rate": ('sys-interrupt-rate', 'delta-s'),
        "disk-io-time": ('disk-io-time', 'delta-s')
    }
    measures = list(measurements.keys())
    
    threshold_results = pd.DataFrame(columns=["Measurement", "Threshold", "Threshold_Value", "Probability_Larger", "Probability_Smaller"])
    #cross_tab_results = pd.DataFrame()
    conditional_prob_results = pd.DataFrame(columns=["Condition", "Probability"])
    # Example placeholders for figures
    fig, ax = plt.subplots(figsize=(10, 10))
    # Placeholder for populating threshold-based probability estimation results
    for measure in measures:
        thresholds = [('mean',df[measure].mean()), ('10%(max)', df[measure].max()*0.1), ('90%(max)',df[measure].max()*0.9)]  # Example thresholds
        for name,threshold in thresholds:
            # Placeholder calculation (replace with actual logic)
            probability_1 = np.mean(df[measure] > threshold)
            probability_2 = 1 - probability_1
            
            threshold = np.round(threshold, decimals=4)
            probability_1 = np.round(probability_1, decimals=4)
            probability_2 = np.round(probability_2, decimals=4)
            threshold_results = pd.concat([
                threshold_results, 
                pd.DataFrame({"Measurement":[measure], "Threshold": [name], "Threshold_Value": [threshold], "Probability_Larger": [probability_1], "Probability_Smaller": [probability_2]})
            ], ignore_index=True)

    # Output threshold results to a figure
    ax.axis('off')  # Turn off axis
    ax.table(cellText=threshold_results.values, colLabels=threshold_results.columns, loc='center')
    ax.set_title("Threshold-based Probability Estimation")
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
        
    fig, ax = plt.subplots(figsize=(8, 6))
    
    crosstable_data = {}

    for measure in measures:
        threshold_mean = df[measure].mean()
        total_count = len(df[measure])
        larger_count = (df[measure] > threshold_mean).sum()
        not_larger_count = total_count - larger_count
        
        crosstable_data[measure] = {
            " x > Mean": larger_count,
            " x < Mean": not_larger_count,
            "Total": total_count
        }

    # Convert to DataFrame for visualization
    crosstable_df = pd.DataFrame(crosstable_data).transpose()
    crosstable_df.index.name = "Measurement"

    # Display the crosstable
    ax.axis('off')  # Hide axis
    ax.table(
        cellText=crosstable_df.values,
        rowLabels=crosstable_df.index,
        colLabels=crosstable_df.columns,
        loc='center'
    )
    ax.set_title("Crosstable Analysis Based on Mean Threshold")
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
        
    fig, ax = plt.subplots(figsize=(10, 20))
    # Generate pairs
    pairs = [(measures[i], measures[j]) for i in range(len(measures)) for j in range(i + 1, len(measures))]

    conditional_prob_results = pd.DataFrame(columns=["Condition", "Probability"])

    # Loop over the pairs
    for i, (measure1, measure2) in enumerate(pairs):
        thresholds = [
            (f"P({measure1} > (mean) | {measure2} > (mean))", lambda: np.mean((df[measure1] > df[measure1].mean()) & (df[measure2] > df[measure2].mean()))),
            (f"P({measure1} < (mean) | {measure2} > (mean))", lambda: np.mean((df[measure1] < df[measure1].mean()) & (df[measure2] > df[measure2].mean()))),
            (f"P({measure1} > 90%(max) | {measure2} > 90%(max))", lambda: np.mean((df[measure1] > df[measure1].max() * 0.9) & (df[measure2] > df[measure2].max() * 0.9))),
            (f"P({measure1} < 10%(max) | {measure2} < 10%(max))", lambda: np.mean((df[measure1] < df[measure1].max() * 0.1) & (df[measure2] < df[measure2].max() * 0.1))),
            (f"P({measure1} < 10%(max) | {measure2} > 90%(max))", lambda: np.mean((df[measure1] < df[measure1].max() * 0.1) & (df[measure2] > df[measure2].max() * 0.9))),
            (f"P({measure2} < 10%(max) | {measure1} > 90%(max))", lambda: np.mean((df[measure2] < df[measure2].max() * 0.1) & (df[measure1] > df[measure1].max() * 0.9))),
            (f"P({measure1} > 90%(max) | {measure2} < 10%(max))", lambda: np.mean((df[measure1] > df[measure1].max() * 0.9) & (df[measure2] < df[measure2].max() * 0.1))),
            (f"P({measure2} > 90%(max) | {measure1} < 10%(max))", lambda: np.mean((df[measure2] > df[measure2].max() * 0.9) & (df[measure1] < df[measure1].max() * 0.1))),
        ]

        for condition, calc in thresholds:
            probability = calc()
            probability = np.round(probability, decimals=4)
            if probability >= 0.001:  # Only add rows with probability >= 0.0001
                conditional_prob_results = pd.concat([
                    conditional_prob_results,
                    pd.DataFrame({"Condition": [condition], "Probability": [probability]})
                ], ignore_index=True)

    # Display or further process the results as needed
    ax.axis('off')
    ax.table(cellText=conditional_prob_results.values, colLabels=conditional_prob_results.columns, loc='center')
    ax.set_title("Conditional Probability Analysis")
    
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=150, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    save_image(image_names, out_dir, out_name)
    
    if not show_plots:
        plt.close("all")

In [ ]:
for file in files:  
    prob_analysis(False, pathProcessd, file, None)

## Analysis
- **2.3 Summary of observations from each task**
> ...
-

## 2.3 Statistical Theory Applications
- **Law of Large Numbers demonstration**
- **Central Limit Theorem application**
- **Result interpretation**
> ...

In [ ]:
#TODO

## 2.3 Regression Analysis
- **Linear/Polynomial model selection**
- **Model fitting and validation**
- **Result interpretation and analysis**
> ...

In [ ]:
#TODO